# Wildlife Camera Trap Classification with Azure Custom Vision
*A Microsoft AI Services Tutorial for Research*

## What This Demonstrates

**Microsoft Custom Vision** for automated wildlife monitoring. We'll show how to:

- Set up an Azure Custom Vision project 
- Train a model with minimal wildlife data
- Use the prediction API for batch processing
- Export results for research analysis

**Why Custom Vision for Research:**
- **No ML expertise required** - visual training interface
- **Small data friendly** - works with as few as 5 images per category  
- **Research-ready APIs** - confidence scores and batch processing
- **Scalable** - handles thousands of images via REST API

**Learning Objectives:**
By the end of this tutorial, you will understand how to create a custom image classification model that can automatically identify wildlife species in camera trap images, demonstrating the practical application of AI in ecological research.

## Setup: Install Required Packages

Before we begin, we need to install the necessary Python packages for working with Azure Custom Vision. These packages provide the tools to communicate with Azure's AI services and handle image processing.

**Follow the steps in the online teaching resource to set up your Azure Custom Vision credentials, then resume your journey in this notebook.**

In [ ]:
# Install required packages for Azure Custom Vision and image processing
!pip install azure-cognitiveservices-vision-customvision
!pip install azure-cognitiveservices-vision-computervision  
!pip install pillow matplotlib pandas requests

## Import Libraries

Now we'll import all the necessary libraries for our wildlife classification project. These include Azure AI services, image processing tools, and data analysis libraries.

**You don't need to understand the details of these imports** - simply run this cell to make the required functionality available.

In [ ]:
# Import libraries for Azure Custom Vision, image processing, and data analysis
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry
from msrest.authentication import ApiKeyCredentials
from PIL import Image as PILImage
import matplotlib.pyplot as plt
import pandas as pd
import requests
import time
import io

## Azure Custom Vision Configuration

This cell establishes the connection to your Azure Custom Vision service using the credentials you obtained from the Azure Portal. 

**Replace the placeholder values below with your actual Azure Custom Vision credentials:**

- **ENDPOINT**: Your Custom Vision service endpoint URL
- **TRAINING_KEY**: API key for training operations  
- **PREDICTION_KEY**: API key for prediction operations
- **PREDICTION_RESOURCE_ID**: Full resource path for predictions

These credentials allow the notebook to communicate securely with your Azure AI services.

In [ ]:
# Azure Custom Vision credentials - replace with your actual values from Azure Portal
ENDPOINT = "https://your-resource.cognitiveservices.azure.com/"
TRAINING_KEY = "your_training_key_here"
PREDICTION_KEY = "your_prediction_key_here" 
PREDICTION_RESOURCE_ID = "/subscriptions/your-sub-id/resourceGroups/your-rg/providers/Microsoft.CognitiveServices/accounts/your-prediction-resource"

# Initialize Azure Custom Vision clients
training_credentials = ApiKeyCredentials(in_headers={"Training-key": TRAINING_KEY})
trainer = CustomVisionTrainingClient(ENDPOINT, training_credentials)

prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

print("Custom Vision clients successfully initialized")
print("Ready to proceed with wildlife classification project")

## Project Setup: Create Wildlife Classification Project

This cell sets up your Custom Vision project and creates the animal categories (tags) that the model will learn to recognize. The function handles existing projects gracefully to avoid rate limit issues.

**What this creates:**
- A Custom Vision project called "Wildlife Camera Trap Classifier"
- 10 classification categories: deer, fox, bear, raccoon, coyote, rabbit, squirrel, bird, empty (no animal), and human

**Important Note:** Run this cell only once. If you run it multiple times, you may hit Azure's free tier rate limits. The function will automatically detect and use existing projects if available.

In [ ]:
def setup_wildlife_project():
    """
    Set up Custom Vision project for wildlife classification.
    Handles existing projects and rate limits gracefully.
    """
    
    print("Setting up Wildlife Classification Project...")
    
    try:
        # Check for existing Wildlife projects to avoid duplicates
        existing_projects = trainer.get_projects()
        wildlife_projects = [p for p in existing_projects if "Wildlife" in p.name]
        
        if wildlife_projects:
            # Use existing project
            project = wildlife_projects[-1]  # Use most recent
            print(f"Using existing project: {project.name}")
            print(f"Project ID: {project.id}")
            
            # Get existing tags from project
            existing_tags = trainer.get_tags(project.id)
            if existing_tags:
                tags = {tag.name: tag for tag in existing_tags}
                print(f"Found {len(tags)} existing categories: {list(tags.keys())}")
                return project, tags
            else:
                print("Project exists but no categories found. Creating categories...")
                
        else:
            # Create new project
            print("Creating new Wildlife Camera Trap Classifier project...")
            project = trainer.create_project(
                "Wildlife Camera Trap Classifier",
                description="Classify animals from camera trap images for ecological research"
            )
            print(f"Project created successfully: {project.id}")
        
        # Create classification categories (tags)
        categories = [
            "deer", "fox", "bear", "raccoon", "coyote", 
            "rabbit", "squirrel", "bird", "empty", "human"
        ]
        
        tags = {}
        print("Setting up classification categories...")
        
        for category in categories:
            tag = trainer.create_tag(project.id, category)
            tags[category] = tag
            print(f"Created category: {category}")
        
        print(f"Setup complete! Ready to train with {len(tags)} categories.")
        return project, tags
        
    except Exception as e:
        if "Too Many Requests" in str(e):
            print("Rate limit reached. Please wait 15-30 minutes and try again.")
            print("Alternatively, create a new Custom Vision resource in Azure Portal.")
        else:
            print(f"Setup error: {e}")
        return None, None

# Execute project setup
project, tags = setup_wildlife_project()

# Verify setup success
if project and tags:
    print(f"\nProject setup successful!")
    print(f"Project Name: {project.name}")
    print(f"Available Categories: {len(tags)} wildlife types")
    print("Ready to proceed with training data upload")
else:
    print("\nSetup incomplete. Please resolve rate limits or check credentials.")

## Training Data: Wildlife Image URLs

This cell defines our training dataset using a Python dictionary. Each category (deer, fox, bear, etc.) is associated with a list of image URLs from Wikimedia Commons - a free, public repository of wildlife photographs.

**How this works:**
- Each key in the dictionary represents an animal category
- Each value is a list of 5-6 image URLs showing that animal
- These images will be downloaded and used to train our AI model
- The model learns to recognize patterns that distinguish each animal type

**For research applications**, you would replace these URLs with images from your own camera trap surveys. Custom Vision works best with 10-50 images per category, representing different angles, lighting conditions, and individual animals.

In [ ]:
# Training dataset: URLs of wildlife images from Wikimedia Commons
# Each category contains 5-6 representative images for model training
image_data = {
    "deer": [
        "https://upload.wikimedia.org/wikipedia/commons/5/52/20190121_Nara_deer-3.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/b/b2/Sika_Deer_in_Nara%2C_Japan%2C_20240819_1546_4782.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/0/02/Baluran%27s_Deer_Family_Chandra_Musliminnata.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/9/9a/20190121_Nara_deer-10.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/3/33/Deer_walking_in_the_snow.jpg"
    ],
    
    "fox": [
        "https://upload.wikimedia.org/wikipedia/commons/0/03/Vulpes_vulpes_laying_in_snow.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/d/d5/Vulpes_vulpes_sitting.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/d/d9/Culpeo_MC.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/c/cc/Portrait_of_a_red_fox_in_Rautas_fj%C3%A4llurskog.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/4/4d/Red_Fox_%28Vixen%29.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/c/c7/Red_fox.jpg"
    ],
    
    "bear": [
        "https://upload.wikimedia.org/wikipedia/commons/5/5d/Kamchatka_Brown_Bear_near_Dvuhyurtochnoe_on_2015-07-23.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/7/7f/European_Brown_Bear.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/5/54/Closeup_kodiak_bear_hamburg.JPG", 
        "https://upload.wikimedia.org/wikipedia/commons/2/2a/Brown_bear_%28Ursus_arctos_arctos%29_running.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/6/6b/Kodiak_Brown_Bear.jpg"
    ],
    
    "raccoon": [
        "https://upload.wikimedia.org/wikipedia/commons/9/99/Raccoon_%28Procyon_lotor%29_3.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/d/dd/Waschb%C3%A4r_Procyon_lotor-7318.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/3/3e/Raccoon_in_Central_Park_%2835264%29.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/f/fe/Procyon_lotor_%28Common_raccoon%29.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/8/8b/Procyon_lotor_%28raccoon%29.jpg"
    ],
    
    "coyote": [
        "https://upload.wikimedia.org/wikipedia/commons/a/af/Canis_latrans.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/2/20/Coyote_Tule_Lake_CA.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/d/db/Mother_Coyote_Sitting%2C_Watching_Over_Den%2C_Photo_1_of_2_%2847924154788%29.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/2/26/Coyote_%28Canis_latrans%29_DSC1747vv.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/4/48/Coyote_%28Canis_latrans%29_DSC2777a.jpg"
    ],
    
    "rabbit": [
        "https://upload.wikimedia.org/wikipedia/commons/f/fa/Bunny_rabbit_at_Alligator_Bay%2C_Beauvoir%2C_France.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/b/b7/Trix_the_rabbit.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/e/ef/Rabbit_in_Richmond_BC.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/8/8c/Rabbit_%28Oryctolagus_cuniculus%29_Skomer.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/3/37/Oryctolagus_cuniculus_Tasmania_2.jpg"
    ], 
    
    "squirrel": [
        "https://upload.wikimedia.org/wikipedia/commons/b/bd/Red_squirrel_%2821808%29.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/0/07/Grey_squirrel_%28Sciurus_carolinensis%29_02.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/7/7c/Eastern_Grey_Squirrel_in_St_James%27s_Park%2C_London_-_Nov_2006_edit.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/0/02/Eichh%C3%B6rnchen_D%C3%BCsseldorf_Hofgarten_edit.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/8/87/Squirrel_eating_nut_in_grass.png"
    ],
    
    "bird": [
        "https://upload.wikimedia.org/wikipedia/commons/f/fb/Brown_thrasher_in_CP_%2802147%29.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/e/e1/Sitta_europaea_wildlife_3.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/2/2e/Crested_Tern_-_Mortimer_Bay.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/3/33/Gyps_rueppellii_-Nairobi_National_Park%2C_Kenya-8-4c.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/e/ec/Ara_ararauna_Luc_Viatour.jpg"
    ],
    
    "empty": [
        "https://upload.wikimedia.org/wikipedia/commons/b/be/Hutan_Tangkahan.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/3/34/D%C3%BClmen%2C_Naturschutzgebiet_-Am_Enteborn-_--_2014_--_0202.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/3/3d/Sossusvlei_south_view.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/9/90/Everest%2C_Himalayas.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/a/a5/Ponta_de_S%C3%A3o_Louren%C3%A7o_north_north_east.jpg"
    ],
    
    "human": [
        "https://upload.wikimedia.org/wikipedia/commons/0/07/Isabella_L%C3%B6vin_signing_climate_law_referral.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/c/cc/People_in_JHB.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/8/89/Folk_Dance%2C_Odisha_India.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/0/0c/Sally_Ride_%281984%29.jpg", 
        "https://upload.wikimedia.org/wikipedia/commons/f/f4/People_of_Butte_County_%E2%80%93_Tod_Kimmelshue_%282025%29-104A8815.jpg"
    ]
}

# Display training data summary
print("Training dataset summary:")
total_images = sum(len(urls) for urls in image_data.values())
print(f"Total categories: {len(image_data)}")
print(f"Total training images: {total_images}")
print("\nImages per category:")
for category, urls in image_data.items():
    print(f"  {category}: {len(urls)} images")

## Upload Training Images to Custom Vision

This cell uploads the training images to your Custom Vision project. The function handles several important technical details:

**Key features:**
- **Automatic image resizing**: Large images are resized to meet Azure's 6MB limit
- **Reliable downloads**: Uses proper headers to avoid being blocked by image servers
- **Error handling**: Continues processing even if some images fail to download
- **Progress tracking**: Shows upload progress for each category

**What happens during upload:**
1. Downloads each image from its URL
2. Resizes oversized images to fit Azure's requirements
3. Associates each image with its correct category (tag)
4. Uploads the processed image to your Custom Vision project

This process typically takes 2-5 minutes depending on your internet connection and the number of images.

In [ ]:
def upload_training_images(project_id, tags, training_data):
    """
    Upload training images to Custom Vision with automatic optimization.
    
    Features:
    - Handles large images (resizes automatically)
    - Proper headers for reliable downloads
    - Progress tracking and error handling
    """
    
    print("Starting training image upload...")
    print(f"Processing {len(training_data)} categories...")
    
    # Configuration for reliable downloads and processing
    headers = {'User-Agent': 'Mozilla/5.0 (compatible; Educational-Bot)'}
    max_dimension = 1024  # Maximum image dimension in pixels
    max_file_size = 6 * 1024 * 1024  # 6MB limit for Custom Vision
    uploaded_count = 0
    
    for category, image_urls in training_data.items():
        if category not in tags:
            print(f"Skipping unknown category: {category}")
            continue
            
        print(f"\nUploading {category} ({len(image_urls)} images)...")
        category_success = 0
        
        for i, url in enumerate(image_urls, 1):
            try:
                # Download image with proper headers
                response = requests.get(url, headers=headers, timeout=15)
                if response.status_code != 200:
                    print(f"  {i}/{len(image_urls)}: Download failed (HTTP {response.status_code})")
                    continue
                
                image_content = response.content
                
                # Resize image if it's too large
                if len(image_content) > max_file_size:
                    img = PILImage.open(io.BytesIO(image_content))
                    img.thumbnail((max_dimension, max_dimension), PILImage.Resampling.LANCZOS)
                    
                    # Convert to RGB if necessary (for JPEG compatibility)
                    if img.mode in ('RGBA', 'P'):
                        img = img.convert('RGB')
                    
                    # Save resized image to bytes
                    img_bytes = io.BytesIO()
                    img.save(img_bytes, format='JPEG', quality=85, optimize=True)
                    image_content = img_bytes.getvalue()
                
                # Create image entry for Custom Vision
                image_entry = ImageFileCreateEntry(
                    name=f"{category}_{i}.jpg",
                    contents=image_content,
                    tag_ids=[tags[category].id]
                )
                
                # Upload to Custom Vision
                result = trainer.create_images_from_files(
                    project_id, ImageFileCreateBatch(images=[image_entry])
                )
                
                if result.is_batch_successful:
                    uploaded_count += 1
                    category_success += 1
                    print(f"  {i}/{len(image_urls)}: Success")
                else:
                    print(f"  {i}/{len(image_urls)}: Upload failed")
                    
            except Exception as e:
                print(f"  {i}/{len(image_urls)}: Error - {str(e)[:50]}...")
        
        print(f"  {category} complete: {category_success}/{len(image_urls)} images uploaded")
    
    print(f"\nUpload complete!")
    print(f"Total images successfully uploaded: {uploaded_count}")
    print("Your Custom Vision project now has training data and is ready for model training.")
    
    return uploaded_count

# Execute the upload process
if project and tags:
    uploaded_images = upload_training_images(project.id, tags, image_data)
    
    if uploaded_images > 0:
        print(f"\nSuccess! {uploaded_images} images uploaded to Custom Vision.")
        print("Proceed to the next section to train your wildlife classification model.")
    else:
        print("\nNo images were uploaded successfully.")
        print("Please check your internet connection and try again.")
else:
    print("Cannot upload images - project setup was not successful.")
    print("Please run the project setup cell first.")

## Section 1 Complete

**Congratulations!** You have successfully completed the setup and data upload phase of the wildlife classification tutorial.

**What you've accomplished:**
- ✅ Set up Azure Custom Vision project with 10 wildlife categories
- ✅ Defined training dataset with representative wildlife images
- ✅ Uploaded training images to Custom Vision service
- ✅ Your AI model now has the data it needs for training

**Next steps (Section 2):**
- Train your custom wildlife classification model
- Test the model on new wildlife images
- Explore batch processing capabilities for research
- Learn about model evaluation and improvement techniques

**Research applications:** This same workflow can be adapted for any wildlife monitoring project by substituting your own camera trap images for the training dataset.